In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Demo').getOrCreate()

22/03/28 21:49:43 WARN Utils: Your hostname, shital-VivoBook-ASUS-Laptop-X505ZA-X505ZA resolves to a loopback address: 127.0.1.1; using 192.168.0.241 instead (on interface wlp1s0)
22/03/28 21:49:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/28 21:49:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv("customer_churn.csv", inferSchema=True, header=True)

In [4]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [6]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assesmbler = VectorAssembler(inputCols=['Age',
'Total_Purchase',
'Account_Manager',
'Years',
'Num_Sites'], outputCol='features')

In [9]:
output = assesmbler.transform(df)

In [10]:
df_final = output.select("features", "churn")

In [11]:
df_final.show()

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [12]:
train, test = df_final.randomSplit([0.7, 0.3], seed=42)

In [13]:
from pyspark.ml.classification import LogisticRegression

In [14]:
lr = LogisticRegression(labelCol='churn')

In [15]:
lrm = lr.fit(train)

22/03/28 21:50:22 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/03/28 21:50:22 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [16]:
lrm_summary = lrm.summary

In [17]:
lrm_summary.predictions.show()

/home/shital/.local/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|  0.0|[4.55979963242689...|[0.98964420919358...|       0.0|
|[25.0,9672.03,0.0...|  0.0|[4.67536757133124...|[0.99076400091638...|       0.0|
|[26.0,8939.61,0.0...|  0.0|[6.28230440567530...|[0.99813439848473...|       0.0|
|[27.0,8628.8,1.0,...|  0.0|[5.32554251161007...|[0.99515784990742...|       0.0|
|[28.0,8670.98,0.0...|  0.0|[7.59026200904823...|[0.99949490661754...|       0.0|
|[28.0,11128.95,1....|  0.0|[4.09749022712612...|[0.98365720318515...|       0.0|
|[29.0,5900.78,1.0...|  0.0|[4.06733742325028...|[0.98316533957374...|       0.0|
|[29.0,8688.17,1.0...|  1.0|[2.71962101743710...|[0.93817455523842...|       0.0|
|[29.0,9378.24,0.0...|  0.0|[4.73007562142384...|[0.99125140974466...|       0.0|
|[29.0,12711.15,

In [18]:
lrm_summary.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               667|                667|
|   mean|0.1634182908545727|0.12293853073463268|
| stddev|0.3700243606477147|0.32861306618408714|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [20]:
pred_labels = lrm.evaluate(test)

In [21]:
pred_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.79106248132019...|[0.68805942038441...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.61026688857554...|[0.83344843709585...|       0.0|
|[28.0,11204.23,0....|    0|[1.97148345642761...|[0.87777036176974...|       0.0|
|[28.0,11245.38,0....|    0|[3.75331012854097...|[0.97709682330471...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.42202807822146...|[0.98813267415840...|       0.0|
|[29.0,10203.18,1....|    0|[3.71080419195966...|[0.97612605863733...|       0.0|
|[29.0,11274.46,1....|    0|[4.39058463869247...|[0.98775823667280...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.55749267043436...|[0.97228008128416...|       0.0|
|[30.0,8403.78,1.0...|    0|[5.76304568829244...|[0.99686830940139...|       0.0|
|[30.0,8874.83,0

In [22]:
eval = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="churn")

In [23]:
auc = eval.evaluate(pred_labels.predictions)

In [24]:
auc

0.7456808943089431